# Vectorestores and Embeddings

Recall the overall workflow for retrieval augmented generation

![](rag.jpeg)

In [1]:
import openai

In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
# Load PDFs
loaders = [
    # Duplicate documents on purpose - dump data
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture03.pdf")
]

In [4]:
docs = []
for loader in loaders:
    docs.extend(loader.load())


In [5]:
# Split the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [6]:
splits = text_spliter.split_documents(docs)

In [7]:
len(splits)

209